## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

In [2]:
# По количеству строк сразу видно, что в колонках days_employed и total_income есть пустые значения
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
# Для начала заполним пустые значения нулями
data['days_employed'] = data['days_employed'].fillna(0)
data['total_income'] = data['total_income'].fillna(0)

In [4]:
# В 5-ой строке таблицы видим days_employed = 340266: число хоть и положительное, но аномально большое. 
# Получается, что трудовой стаж человека - почти 931 год!
# Проверка показывает, что все положительные значения в колонке days_employed аномальные:
print(data[data['days_employed'] > 0]['days_employed'].min())
print(data[data['days_employed'] > 0]['days_employed'].shape[0])

328728.72060451825
3445


In [5]:
# Принял, решение обнулить все положительные значения в колонке days_employed
# !!!!!!!!!!!!!!!!!!!!!!!!
# Первоначально замену пустых значений делал вот так, но постоянно возникала ошибка:
# A value is trying to be set on a copy of a slice from a DataFrame
# data[data['days_employed'] > 0]['days_employed'] = 0
# Буду очень признателен, если подскажите как эту ошибку надо было обойти 

def get_zero_for_positive_days_employed(days_employed):
    if days_employed > 0:
        return 0
    else: return days_employed
    
data['days_employed'] = data['days_employed'].apply(get_zero_for_positive_days_employed)

print(data[data['days_employed'] > 0].shape[0])

0


In [6]:
# Также замечаем отрицательные значения в колонке days_employed. 
# Они хоть и отрицательные, но не такие экстремально большие
print(data[data['days_employed'] < 0]['days_employed'].max())
print(data[data['days_employed'] < 0]['days_employed'].min())

-24.14163324048118
-18388.949900568383


In [7]:
# Сейчас предположу, что знак можно заменить с минуса на плюс, это и будет "правильным" значением.
# Чуть дальше мы проверим "адекватность" такого предположения.
data['days_employed'] = abs(data['days_employed'])

# В колонке total_income все числа положительные
print(data['total_income'].min())
print(data['total_income'].max())

0.0
2265604.028722744


In [8]:
# Заменять пропущенные значения стажа и дохода средними некорректно: 
# все-таки они сильно зависят от возраста и пола человека. 
# Поэтому в первую очередь займемся изучением данных в столбцах возраст и пол
print(data['dob_years'].value_counts().sort_index())
print(data['gender'].value_counts())

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64
F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [9]:
# Обнаружили, что есть 101 запись с 0 возрастом и 1 запись без указания пола.
# Посмотрел внимательно на проблемные записи. Пришел к выводу, что эти данные нам не восстановить.
# Но, к счастью, строк немного - удалим их из нашей таблицы:
data = data[(data['dob_years'] != 0) & (data['gender'] != 'XNA')].reset_index(drop=True)

Для возраста и пола предполагаю <b>человеческий фактор</b> в появлении ошибок. Система позволила не указать требуемые параметры - они остались пустыми.

In [10]:
# Теперь, когда мы уверены в полноте данных о возрастах и полах клиентов, 
# займемся заполнением пропусков в данных о стаже и доходе

# Категоризируем людей по возрастам (0-20, 21-30, 31-40, 41-50, 50-60, 60+)
def get_dob_years_interval(dob_years):
    if dob_years <= 20:
        return '0-20'
    elif 20 < dob_years <= 30:
        return '21-30'
    elif 30 < dob_years <= 40:
        return '31-40'
    elif 40 < dob_years <= 50:
        return '41-50'
    elif 50 < dob_years <= 60:
        return '51-60'
    else: return '60+'

data_wo_mistakes = data[(data['days_employed'] > 0) & (data['total_income'] > 0)].reset_index(drop=True)
data_wo_mistakes['dob_years_interval'] = data_wo_mistakes['dob_years'].apply(get_dob_years_interval)

# Формируем итоговую сводную таблицу, в которой для каждой пары (gender, dob_years_interval)
# будут храниться среднее значение days_employed и total_income
days_employed_income_pivot = data_wo_mistakes.pivot_table(
    index=['gender', 'dob_years_interval'], 
    values=['days_employed', 'total_income'], 
    aggfunc={'days_employed': ['count', 'mean'], 'total_income': 'mean'})

# Для простоты дальнейшей работы:
# убираем "сводность" таблицы
# меняем имена колонок на немультииндексные
days_employed_income = days_employed_income_pivot.reset_index()
days_employed_income.set_axis(['gender', 'dob_years_interval', 'clients', 'days_employed_mean', 'total_income_mean'],
                             axis='columns',
                             inplace=True)

print(days_employed_income)

   gender dob_years_interval  clients  days_employed_mean  total_income_mean
0       F               0-20       35          619.910256      114503.073807
1       F              21-30     1916         1315.458540      147374.784105
2       F              31-40     3145         2136.344388      160978.856909
3       F              41-50     2993         3006.786963      162029.060595
4       F              51-60     1551         3568.865959      166322.462419
5       F                60+      257         4419.381813      180063.837421
6       M               0-20       24          752.016431      133951.744304
7       M              21-30     1383         1255.193186      182712.592052
8       M              31-40     1988         2018.572495      204022.982352
9       M              41-50     1579         2438.848269      207991.318586
10      M              51-60      821         2820.656468      190744.210556
11      M                60+      139         2855.811556      192540.217711

Ну что сказать - данные о стаже days_employed мягко говоря "битые": вряд ли 60+ летние мужчины имеют средний стаж 8 лет.Безусловно тут требуется контакт с отделом отвечающим за выгрузку данных. С учетом наличия и отрицательных данных в этой колонке - предполагаю <b>технологический характер</b> ошибки (проблемы при выгрузке).

In [11]:
# И теперь заменяем пустые значения в data
# Для этого присоединяем к таблице data таблицу days_employed_income и в итоговой таблице заменяем пустые значения: 
#     в колонке days_employed значением из колонки days_employed_mean сводной таблицы
#     в колонке total_income значением из колонки total_income_mean сводной таблицы
data['dob_years_interval'] = data['dob_years'].apply(get_dob_years_interval)

data = data.merge(days_employed_income, on=['gender', 'dob_years_interval'], how='left')

# !!!!!!!!!!!!!!!!!!!!!!!!
# Первоначально замену пустых значений делал вот так, но постоянно возникала ошибка:
# A value is trying to be set on a copy of a slice from a DataFrame.
# data[data['days_employed'] == 0]['days_employed'] = data['days_employed_mean']
# data[data['total_income'] == 0]['total_income'] = data['total_income_mean']
# Буду очень признателен, если подскажите как эту ошибку надо было обойти 
# (кроме варианта создания нового df: data1 = data.merge...)

def get_days_employed(row):
    if row['days_employed'] == 0:
        return row['days_employed_mean']
    else: return row['days_employed']

def get_total_income(row):
    if row['total_income'] == 0:
        return row['total_income_mean']
    else: return row['total_income']

data['days_employed'] = data.apply(get_days_employed, axis=1)
data['total_income'] = data.apply(get_total_income, axis=1)

# Удаляем лишние колонки
data.drop(['clients', 'days_employed_mean', 'total_income_mean'], axis='columns', inplace=True)

# Проверяем, что все значения заполнились
print(data[(data['days_employed'] == 0) & (data['total_income'] == 0)].shape[0])

0


In [12]:
# Проверяем остальные колонки на предмет артефактов

# children
print(data['children'].unique())

# В количестве детей присутствует несколько записей с -1 и 20 детьми. Таких записей немного:
print('children - строк с ошибками', data[data['children'] == -1].shape[0])
print('children - строк с ошибками', data[data['children'] == 20].shape[0])

[ 1  0  3  2 -1  4 20  5]
children - строк с ошибками 47
children - строк с ошибками 75


Для поля children предполагаю <b>технологический фактор</b> в появлении ошибок - необходимо связаться с отделом, предоставившим файл выгрузки. С трудом верится, что 75 клиентов указали ровно 20 детей (почему не 100, например).

In [13]:
# Принял решение заменить значения в этих строках медианным значением. 
# Если оно окажется дробным - округляем до целого по математическим правилам
children_median = data[(data['children'] != -1) & (data['children'] != 20)]['children'].median()
print('Количество детей медиана:', children_median)

data.loc[(data['children'] == -1) | (data['children'] == 20), 'children'] = int(children_median)

print(data['children'].unique())

Количество детей медиана: 0.0
[1 0 3 2 4 5]


In [14]:
# education, education_id
education_dict = data[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True).sort_values(by='education_id').reset_index(drop=True)
education_dict
# Все в порядке, лемматизацию проведем позднее

,education,education_id
0,высшее,0
1,ВЫСШЕЕ,0
2,Высшее,0
3,среднее,1
4,Среднее,1
5,СРЕДНЕЕ,1
6,неоконченное высшее,2
7,НЕОКОНЧЕННОЕ ВЫСШЕЕ,2
8,Неоконченное высшее,2
9,начальное,3


In [15]:
# family_status, family_status_id
family_status_dict = data[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True).sort_values(by='family_status_id').reset_index(drop=True)
family_status_dict
# Все в порядке

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [16]:
# income_type, debt, purpose
print(data['income_type'].unique())
print(data['debt'].unique())
print(data['purpose'].unique())
# Все в порядке

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
[0 1]
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья

### Вывод

Мы изучили предоставленный файл:
* были удалены строки с незаполненными возрастами и полом
* были восстановлены незаполненные данные о трудовом стаже и ежемесячном доходе
* были заменены артефактные значения о количестве детей
* все остальные колонки были проверены на полноту информации

### Замена типа данных

In [17]:
# Повторно выведем информацию о колонках таблицы:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21422
Data columns (total 13 columns):
children              21423 non-null int64
days_employed         21423 non-null float64
dob_years             21423 non-null int64
education             21423 non-null object
education_id          21423 non-null int64
family_status         21423 non-null object
family_status_id      21423 non-null int64
gender                21423 non-null object
income_type           21423 non-null object
debt                  21423 non-null int64
total_income          21423 non-null float64
purpose               21423 non-null object
dob_years_interval    21423 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 2.3+ MB
None


In [18]:
# Очевидно, что тип данных в колонках days_employed и total_income может быть заменен с float на int
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21422
Data columns (total 13 columns):
children              21423 non-null int64
days_employed         21423 non-null int64
dob_years             21423 non-null int64
education             21423 non-null object
education_id          21423 non-null int64
family_status         21423 non-null object
family_status_id      21423 non-null int64
gender                21423 non-null object
income_type           21423 non-null object
debt                  21423 non-null int64
total_income          21423 non-null int64
purpose               21423 non-null object
dob_years_interval    21423 non-null object
dtypes: int64(7), object(6)
memory usage: 2.3+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_interval
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,41-50
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,31-40
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,31-40
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,31-40
4,0,3568,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,51-60


### Обработка дубликатов

In [19]:
# Ранее мы создали dataframe-справочник education_dict
# В нем в колонке education присутствуют дубли,
# от них можно избавиться, если привести значения к нижнему регистру
education_dict['education'] = education_dict['education'].str.lower()
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [20]:
# Поскольку мы создали отдельный словарь, то в таблице data можно оставить только колонку education_id
# Также у нас есть отдельный словать family_status_dict, и он без дублей. Из таблицы data удалим колонку family_status
data = data.drop(columns=['education', 'family_status'], axis=1)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21423 entries, 0 to 21422
Data columns (total 11 columns):
children              21423 non-null int64
days_employed         21423 non-null int64
dob_years             21423 non-null int64
education_id          21423 non-null int64
family_status_id      21423 non-null int64
gender                21423 non-null object
income_type           21423 non-null object
debt                  21423 non-null int64
total_income          21423 non-null int64
purpose               21423 non-null object
dob_years_interval    21423 non-null object
dtypes: int64(7), object(4)
memory usage: 2.0+ MB
None


### Вывод

Мы удалили дубли из созданных ранее словарей education_dict и family_status_dict используя приведение к нижнему регистру.
В связи с использованием словарей мы удалили избыточную информацию из таблицы data (колонки education и family_status).

### Лемматизация

In [21]:
# В колонке purpose имеются явные дубли, которые нужно привести к общему знаменателю. 
# Основными целями получения кредитов - являются покупка недвижимости, автомобиля и т.п.,
# но все это описано разными словами.
# Используем лемматизацию
from pymystem3 import Mystem
m = Mystem()

def get_purpose_essence(purpose):
    lemmas = m.lemmatize(purpose)
    if ('жилье' in lemmas) or ('недвижимость' in lemmas):
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    else: return purpose

data['purpose_essence'] = data['purpose'].apply(get_purpose_essence)
print(data['purpose_essence'].unique())

['недвижимость' 'автомобиль' 'образование' 'свадьба']


### Вывод

С использованием лемматизации мы выделили главную цель получения кредита

### Категоризация данных

Категоризацию <b>по возрастным группам</b> (колонка dob_years_interval) использовали ранее для заполнения пропущенных значений в стаже и доходе.

Далее будет задан вопрос о зависимости между уровнем дохода и возвратом кредита в срок. Категоризацию таблицы <b>по уровню дохода</b> мы еще не проводили.

In [22]:
# Категоризация по уровню дохода
# Используя квантили подберем оптимальное разделение по уровню дохода, чтобы категоризация была показательной
import numpy as np
print(np.quantile(data['total_income'], 0.2))
print(np.quantile(data['total_income'], 0.4))
print(np.quantile(data['total_income'], 0.6))
print(np.quantile(data['total_income'], 0.8))

98686.40000000002
135618.2
166322.0
214193.00000000006


In [23]:
# Полученные значения немного округлим для удобства
def get_total_income_interval(total_income):
    if total_income <= 100000:
        return '0-100'
    elif 100000 < total_income <= 135000:
        return '100-135'
    elif 135000 < total_income <= 165000:
        return '135-165'
    elif 165000 < total_income <= 215000:
        return '165-215'
    else: return '215+'
        
data['total_income_interval'] = data['total_income'].apply(get_total_income_interval)
print(data.groupby('total_income_interval')['total_income_interval'].count())

total_income_interval
0-100      4440
100-135    4062
135-165    4021
165-215    4654
215+       4246
Name: total_income_interval, dtype: int64


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [25]:
# Заметим, что колонка debt принимающая значение от 0 до 1 станет удачной метрикой при подведении итогов. 
# Когда мы будем рассчитывать среднее значение debt по разным группам, то усредненный debt станет метрикой
# для определения зависимости между выбранной категоризацией и возвратом кредита в срок:
# чем ближе debt к 1, тем чаще данная группа клиентов имеет задолженность по возврату кредитов.

data_children_debt = data.groupby('children')['debt'].mean().sort_values()
print(data_children_debt)

children
5    0.000000
0    0.075136
3    0.082317
1    0.091837
2    0.095005
4    0.097561
Name: debt, dtype: float64


In [26]:
print(data.groupby('children')['children'].count().sort_index())

children
0    14201
1     4802
2     2042
3      328
4       41
5        9
Name: children, dtype: int64


### Вывод

<b>Заемщики с детьми чаще имеют задолженность по возврату кредитов</b>. Причем, чем больше детей, тем чаще просрочка. Правда почему-то заемщики с 5-ю детьми всегда все отдают в срок! Вероятно это из-за слишком маленькой и, поэтому, непоказательной выборки.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
data_family_status_debt = data.groupby('family_status_id')['debt'].mean().to_frame()
data_family_status_debt = data_family_status_debt.merge(family_status_dict, on='family_status_id', how='left').sort_values(by='debt')
print(data_family_status_debt)

   family_status_id      debt          family_status
2                 2  0.064921         вдовец / вдова
3                 3  0.071730              в разводе
0                 0  0.075176        женат / замужем
1                 1  0.092900       гражданский брак
4                 4  0.097605  Не женат / не замужем


### Вывод

Результаты ожидаемые:
* <b>вдовцы реже имеют задолженность</b>: наверное, среди них больше пожилых (и ответственных) людей?
* <b>хуже всего отдают долги не женатые</b>: очевидно, семья учит ответственности

In [28]:
# Действительно: вдовцы - чаще пожилые люди:
print(data[data['family_status_id'] == 2]['dob_years'].mean())

56.809424083769635


In [30]:
# Дополнительно проверим зависимость между возрастом и возвратом кредита в срок
data_dob_years_interval_debt = data.groupby('dob_years_interval')['debt'].mean().sort_index()
print(data_dob_years_interval_debt)

dob_years_interval
0-20     0.076923
21-30    0.108832
31-40    0.095072
41-50    0.076384
51-60    0.061647
60+      0.047196
Name: debt, dtype: float64


Ну а тут прямо нормальное распределение))) И да - <b>пожилые лучше отдают кредиты</b>.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
total_income_interval_debt = data.groupby('total_income_interval')['debt'].mean().to_frame().sort_values(by='debt')
print(total_income_interval_debt)

                           debt
total_income_interval          
215+                   0.070184
0-100                  0.079505
100-135                0.083210
135-165                0.085302
165-215                0.086162


### Вывод

<b>Кредит лучше отдают "бедные" люди и "самые богатые"</b>: и в том и в другом случае - "деньги любят счет"

- Как разные цели кредита влияют на его возврат в срок?

In [32]:
purpose_essence_debt = data.groupby('purpose_essence')['debt'].mean().to_frame().sort_values(by='debt')
print(purpose_essence_debt)

                     debt
purpose_essence          
недвижимость     0.072183
свадьба          0.078835
образование      0.092408
автомобиль       0.093175


### Вывод

<b>По ипотеке реже всего возникает задолженность</b>, несмотря на большую сумму кредита. Причина ясна: жилье очень важная покупка - не платишь и негде жить!

### Пол заемщика

Для полноты картины определим зависимость от пола заемщика

In [35]:
gender_debt = data.groupby('gender')['debt'].mean().to_frame().sort_values(by='debt')
print(gender_debt)

            debt
gender          
F       0.069825
M       0.102493


### Шаг 4. Общий вывод

Был проведен анализ статистики платежеспособности клиентов банка.

В предоставленном файле выявлены следующие проблемы: 
* в колонке <code>days_employed</code> явно присутствуют ошибочные данные, ошибка предположительно имеет технологический характер
* в колонке <code>children</code> присутствуют артефактные значения -1 и 20

Проблемы должны быть переданы сотруднику, отвечающему за выгрузку файла.

Изначальными гипотезами, которые нужно было проверить: <i>"влияет ли семейное положение и количество детей клиента
на факт возврата кредита в срок?"</i>. В ходе исследований выяснили - да, влияет. Действительно:
* чем больше детей у заемщика, тем чаще он имеет задолженность по кредитам (только 7,5% бездетных клиентов имели задолженность, с появлением детей этот показатель рос и достиг 9,7% для клиентов с 4-мя детьми)
* заметно меньше задолженности у клиентов, которые женаты/замужем (или были женаты/замужем ранее) (6,4-7,5% для разных категорий); клиенты, живущие в гражданском браке и холостые чаще имеют задолженность по кредитам (9,2% и 9,7% соответственно)

Дополнительно были получены следующие результаты:
* люди старшего возраста лучше возвращают кредиты (60 и старше - 4,7%)
* реже всего возникает задолженность по ипотечным кредитам, чаще всего кредиту на автомобиль (7,2% против 9,3%)
* женщины гораздо более ответствены при возврате кредитов (7% против 10,2%)

Данный анализ можно использовать при построении модели кредитного скоринга.